# 🎯 インタビュー分析レポート生成ツール

このノートブックは、複数のインタビューセッションを横断的に分析し、仮説検証レポートを生成します。

**元のTypeScriptコード:** `20251123_test-format4-ai-plan-hypothesis.ts`

---

In [ ]:
#@title 📖 使い方ガイド

from IPython.display import Markdown, display

guide = """
# 🎓 このノートブックの使い方

## ステップ1: APIキーの設定 🔑

1. 左サイドバーの **🔑 (Secrets)** アイコンをクリック
2. 「Add new secret」をクリック
3. Name: `GOOGLE_API_KEY`
4. Value: あなたの Google API Key を貼り付け
5. 「Notebook access」をONにする

> **APIキーの取得方法:**  
> [Google AI Studio](https://aistudio.google.com/app/apikey) にアクセスして「Create API Key」

---

## ステップ2: セルを順番に実行 ▶️

1. **📦 環境セットアップ** → ライブラリをインストール
2. **📁 CSVファイルのアップロード** → データをアップロード
3. **⚙️ 設定パラメータ** → モデルやslugを設定
4. **📝 プロンプトテンプレート** (3つのセル) → 必要なら編集
5. **🛠️ ヘルパー関数** → 関数を定義
6. **🔄 処理モード関数** → Sequential/Parallel処理を定義
7. **▶️ メイン実行** → 実際の分析を実行

---

## ステップ3: プロンプトのカスタマイズ（オプション）✏️

プロンプトテンプレートのセル（セル4-6）を直接編集できます。

- **セル4**: 初回レポート生成プロンプト
- **セル5**: 更新プロンプト（逐次処理用）
- **セル6**: マージプロンプト（並列処理用）

編集後、そのセルを再実行してください。

---

## トラブルシューティング 🔧

### APIキーエラー
```
❌ Google API Key の取得に失敗しました
```
→ Secrets に `GOOGLE_API_KEY` が登録されているか確認

### CSVファイルが見つからない
```
⚠️ CSVファイルが見つかりません
```
→ `MESSAGES_CSV_PATH` のパスを修正

### メモリ不足
→ より小さい `TOKENS_PER_BATCH` を試すか、`MIN_MESSAGES` を増やしてセッション数を減らす

---

**準備ができたら、次のセルから順番に実行してください！** 🚀
"""

display(Markdown(guide))

In [ ]:
#@title 📦 環境セットアップ

print("📦 必要なライブラリをインストール中...")

# Google Generative AI SDK をインストール
!pip install -q google-generativeai pandas

print("✅ インストール完了\n")

# インポート
import google.generativeai as genai
import pandas as pd
import json
import os
from datetime import datetime
from typing import List, Dict, Any
from google.colab import files, userdata
import time
from IPython.display import display, HTML, Markdown

print("✅ モジュールのインポート完了")

In [ ]:
#@title 📁 CSVファイルのアップロード

#@markdown 以下の3つの方法から選んでください：

upload_method = "アップロード" #@param ["アップロード", "Google Drive", "すでに配置済み"]

if upload_method == "アップロード":
    print("📤 CSVファイルをアップロードしてください...")
    uploaded = files.upload()
    
    # data/ ディレクトリを作成
    os.makedirs('/content/data', exist_ok=True)
    
    # アップロードされたファイルを保存
    for filename in uploaded.keys():
        if 'messages' in filename:
            target_path = f'/content/data/{filename}'
            with open(target_path, 'wb') as f:
                f.write(uploaded[filename])
            print(f"✅ {target_path} に保存しました")

elif upload_method == "Google Drive":
    from google.colab import drive
    
    print("📂 Google Drive をマウント中...")
    drive.mount('/content/drive')
    
    print("✅ マウント完了")
    print("💡 次のセルで MESSAGES_CSV_PATH を正しいパスに修正してください")
    print("   例: /content/drive/MyDrive/data/bill-of-lading_messages.csv")

else:
    print("✅ ファイルがすでに配置されていることを前提とします")
    print("💡 次のセルで MESSAGES_CSV_PATH が正しいか確認してください")

In [ ]:
#@title ⚙️ 設定パラメータ（初心者向け）

from google.colab import userdata

#@markdown ---
#@markdown ### 🔑 API設定
#@markdown Google APIキーは Colab の「🔑 Secrets」から読み込みます
#@markdown （左サイドバーの鍵アイコン → `GOOGLE_API_KEY` という名前でAPIキーを登録）

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("✅ Google API Key を Colab Secrets から取得しました")
except Exception as e:
    print("❌ Google API Key の取得に失敗しました")
    print("💡 左サイドバーの🔑アイコンから 'GOOGLE_API_KEY' を登録してください")
    raise e

#@markdown ---
#@markdown ### 🤖 モデル設定
MODEL_NAME = "gemini-flash-lite-latest" #@param ["gemini-flash-lite-latest", "models/gemini-3-pro-preview", "gemini-2.0-flash-exp", "gemini-1.5-pro", "gemini-1.5-flash"] {allow-input: true}
MAX_OUTPUT_TOKENS = 64000 #@param {type:"integer"}
TEMPERATURE = 0.3 #@param {type:"slider", min:0, max:1, step:0.1}

#@markdown ---
#@markdown ### 📂 データ設定
slug = "bill-of-lading" #@param {type:"string"}
MIN_MESSAGES = 7 #@param {type:"integer"}
#@markdown ↑ セッションに含まれる最小メッセージ数（これ未満は除外）

MESSAGES_CSV_PATH = "/content/data/bill-of-lading_messages.csv" #@param {type:"string"}
#@markdown ↑ messages.csv のパス

#@markdown ---
#@markdown ### 🔄 処理モード
mode = "sequential" #@param ["sequential", "parallel"]
#@markdown - **sequential**: 逐次処理（バッチを1つずつ処理）
#@markdown - **parallel**: 並列処理（全バッチ同時処理後にマージ）

#@markdown ---
#@markdown ### ⚡ バッチ設定
TOKENS_PER_BATCH = 700000 #@param {type:"integer"}
ESTIMATED_PROMPT_OVERHEAD = 100000 #@param {type:"integer"}

#@markdown ---
#@markdown ### 📝 メモ（オプション）
memo = "" #@param {type:"string"}

# ファイル存在確認
import os
if os.path.exists(MESSAGES_CSV_PATH):
    print(f"✅ CSVファイルが見つかりました: {MESSAGES_CSV_PATH}")
else:
    print(f"⚠️  CSVファイルが見つかりません: {MESSAGES_CSV_PATH}")
    print("💡 ファイルをアップロードするか、パスを修正してください")

print("\n📋 設定内容:")
print(f"   モデル: {MODEL_NAME}")
print(f"   Slug: {slug}")
print(f"   最小メッセージ数: {MIN_MESSAGES}")
print(f"   モード: {mode}")
print(f"   温度: {TEMPERATURE}")

In [ ]:
#@title 📝 プロンプト1: 初回レポート生成（編集可能）

INITIAL_PROMPT_TEMPLATE = """あなたは複数のインタビューセッションを分析し、横断的な考察を行う専門家です。
下記の情報を元に、インタビューのまとめレポートを作成してください。

## インタビューの背景

以下は「{{interviewTitle}}」というテーマで実施されたインタビューの分析依頼です。

【説明】
{{interviewDescription}}

【概要】
{{interviewOverview}}

【主なテーマ】
{{interviewThemes}}

【インタビューで扱った質問】
{{interviewQuestions}}

【参考知識・コンテキスト】
{{knowledgeContext}}

このインタビューは上記の背景、質問、参考知識に基づいて実施されました。この文脈を踏まえて、以下のセッションデータを分析してください。

---

## 検証対象の仮説

以下の「人工知能基本計画に対する提言」における主要な仮説・提案について、インタビューデータから支持する意見と反論・異なる視点を抽出し、検証してください：

【チームみらいの提案・仮説】

仮説1: アジリティを担保する仕組み作りの必要性
AI技術は3ヶ月ごとに状況が変化するため、従来の単年度予算・詳細仕様固定型のIT調達では技術革新に対応できない。複数年度予算枠、3ヶ月ごとの投資領域見直し、「スキャン>パイロット>スケール」方式の採用が必要である。

仮説2: Japan AI Safety Institute（AISI）の育成の重要性
AI安全性評価・標準化・公共調達評価を担うAISIを拡大し、民間AI企業相当の報酬体系で優秀な人材を確保すべきである。AISIが政府のAI利用、AIコモンズ整備、認証制度の基礎を主導することが日本のAI戦略に不可欠である。

仮説3: 政府主導のデータセット整備（AIコモンズ）の必要性
日本版「AIコモンズ」を創設し、NHK等のアーカイブデータ、行政データ、研究機関データを機械可読形式で整備・公開し、国内AI事業者に優先利用させることで国内AI開発を活性化できる。

仮説4: 国内コンテンツ保護とAI規制のバランス
AI事業者にアウトプットが著作権侵害しないための技術的ガードレール設置を課すべきだが、海外と比べ過度な規制は日本のAI事業者の発展を妨げる。国際動向と整合性を取りつつ段階的・漸進的アプローチが必要である。

仮説5: ディープフェイク対策技術への投資の重要性
ディープフェイク判定技術、生成画像の実在判定技術への投資が、悪用取締と表現の自由のバランスを取る上で重要である。技術的判定手段があれば実態に応じた量刑判断が可能になる。

仮説6: 自動運転領域への政府主導支援の必要性
2027年までに全政令指定都市で無人自動運転車を走らせる目標を設定し、行政が積極的に路車協調システム導入や道路再建を行うことで、国内自動運転市場を活性化できる。国産技術開発支援と国際標準化への関与が日本の競争力維持に不可欠である。

仮説7: 年間投資規模・数値目標設定の重要性
ドイツ（年間約2,840億円）をベンチマークに日本のAI投資を50%増加させ、計算資源（GPU規模）、AI人材育成（新規博士号取得者数、AI教授職数）の定量目標を設定することで、省庁横断の目標一致と事後検証が可能になる。


## あなたのタスク

下記のインタビューセッション（{{sessionCount}}セッション）を横断的に分析し、上記の仮説ごとに支持する意見と反論・異なる視点を抽出してMarkdownレポートを生成してください。

注意: これは初回レポート生成です。後ほど追加のセッションデータを統合する予定なので、現時点でのデータに基づいた分析を提供してください。

## 出力フォーマット

# {{interviewTitle}} - 集約レポート

## まとめ

[全体を通じて見えてきた主要なテーマ、パターン、重要な発見を2段落で要約してください。]

## 仮説1: [事前に設定した仮説]

### 支持する意見
- [意見1]
- [意見2]

### 反論・異なる視点
- [反論1]
- [反論2]

### 検証結果
[この仮説に対する総合的な評価]

## 仮説2: [事前に設定した仮説]

[同様に、事前に設定した仮説ごとに章立てして検証結果を記述してください]

---

重要な注意事項:
1. Markdownフォーマットで出力してください
2. HTMLタグは使用しないでください
3. 太字マークダウン（アスタリスク2つ）は使用しないでください。日本語では括弧の処理がうまくいかないことが多いためです
4. 前置きや説明は不要で、「# {{interviewTitle}} - 集約レポート」から始めてください
5. 具体的な発言を引用する場合は、必ず「#」を付けたセッション番号で出典を明記してください
6. 個々のセッションの要約ではなく、横断的な分析に焦点を当ててください
7. 【重要】インタビュー参加者の生の声を積極的に引用してください。「\"発言内容\"(#セッション番号)」の形式で多数引用することで、リアリティと説得力のあるレポートにしてください
8. 【重要】インタビュー対象者の数量的情報や量的表現は一切含めないでください。以下のような表現は使用禁止です：
   - 禁止例: 「多くの参加者が」「一部の人が」「ほとんどの人が」「大半が」「少数だが」
   - 推奨: 「ある参加者は」「〜という意見がある」「〜という視点も示された」など
9. 【重要】価値判断や評価を含む表現を避けてください。以下のような表現は使用禁止です：
   - 禁止例: 「さらに踏み込み」「より高度な」「深い洞察」「優れた提案」「重要度が高い」
   - 推奨: 「〜という意見がある」「〜という視点も示された」など、中立的な記述を使用
10. 【重要】レトリックや主観的な修飾語を排除し、学術論文のように客観的かつ簡潔に記述してください
11. 【重要】表面的な議論にとどまらないように、意見に対する反論や再反論なども積極的に構造化しながら取り上げてください
12. {{outputLengthGuidance}}

---

## セッションデータ ({{sessionCount}}セッション)

{{sessionsData}}
"""

print(f"✅ INITIAL_PROMPT_TEMPLATE 読み込み完了 ({len(INITIAL_PROMPT_TEMPLATE):,} 文字)")

In [ ]:
#@title 📝 プロンプト2: 更新プロンプト（編集可能）

UPDATE_PROMPT_TEMPLATE = """あなたは複数のインタビューセッションを分析し、横断的な考察を行う専門家です。

## タスク概要

以前に作成した集約レポートに、**追加のセッションデータ（{{newSessionCount}}セッション）を統合**して、レポートを更新してください。

**重要**: 最終的なレポートには、どのデータが「新しく追加された」か、「前回のレポートから」といった記述は一切含めないでください。すべてのセッションを一度に分析したかのように、一つの統合されたレポートとして出力してください。

## インタビューの背景

以下は「{{interviewTitle}}」というテーマで実施されたインタビューの分析です。

【説明】
{{interviewDescription}}

【概要】
{{interviewOverview}}

【主なテーマ】
{{interviewThemes}}

【インタビューで扱った質問】
{{interviewQuestions}}

【参考知識・コンテキスト】
{{knowledgeContext}}

---

## 前回のレポート

以下は、これまでに分析したセッションから作成したレポートです：

{{previousReport}}

---

## 新しいセッションデータ ({{newSessionCount}}セッション)

以下の新しいセッションを分析し、上記のレポートに統合してください：

{{sessionsData}}

---

## あなたのタスク

1. 追加のセッションデータを分析し、前回のレポートに含まれていない発見、視点、パターンを特定してください
2. 前回のレポートを更新してください：
   - 新しい発見があれば、該当するカテゴリに追加
   - 新しいカテゴリが必要であれば追加
   - 既存の記述が追加データで補強される場合は、引用を追加
   - 「まとめ」セクションも追加データを反映して更新
3. 前回のレポートの良い部分は維持してください（不要な削除や書き換えは避ける）

**絶対に守るべきルール:**
- 「新たなセッションデータにより」「追加のデータから」「前回のレポートでは」といった、分析プロセスを示唆する表現は一切使用しないでください
- 「より具体的な」「さらに詳細な」など、レポートの更新履歴を示す表現も避けてください
- すべてのセッションを最初から一度に分析したかのように記述してください
- 引用を追加する場合も、必ず「#」を付けたセッション番号で出典を明記してください
- 前回のレポートの構造とスタイルを維持してください
- 前置きや説明は不要で、更新された「# {{interviewTitle}} - 集約レポート」を直接出力してください

注意事項:
1. Markdownフォーマットで出力してください
2. HTMLタグは使用しないでください
3. 太字マークダウン（アスタリスク2つ）は使用しないでください。日本語では括弧の処理がうまくいかないことが多いためです
4. 具体的な発言を引用する場合は、必ず「#」を付けたセッション番号で出典を明記してください
5. 【重要】インタビュー参加者の生の声を積極的に引用してください。「\"発言内容\"(#セッション番号)」の形式で引用してください
6. 【重要】数量的情報や量的表現は一切含めないでください (「多くの参加者が」「一部の人が」など禁止)
7. 【重要】価値判断や評価を含む表現を避けてください (「深い洞察」「優れた提案」など禁止)
8. 【重要】レトリックや主観的な修飾語を排除し、学術論文のように客観的かつ簡潔に記述してください
9. {{outputLengthGuidance}}

---

更新されたレポートを出力してください：
"""

print(f"✅ UPDATE_PROMPT_TEMPLATE 読み込み完了 ({len(UPDATE_PROMPT_TEMPLATE):,} 文字)")

In [ ]:
#@title 📝 プロンプト3: マージプロンプト（編集可能）

MERGE_PROMPT_TEMPLATE = """あなたは複数のインタビューセッションを分析し、横断的な考察を行う専門家です。

## タスク概要

複数のバッチに分けて並列に分析されたインタビューレポートを、**一つの統合されたレポート**にマージしてください。

## インタビューの背景

以下は「{{interviewTitle}}」というテーマで実施されたインタビューの分析です。

【説明】
{{interviewDescription}}

【概要】
{{interviewOverview}}

【主なテーマ】
{{interviewThemes}}

【インタビューで扱った質問】
{{interviewQuestions}}

【参考知識・コンテキスト】
{{knowledgeContext}}

---

## バッチレポート ({{batchCount}}個のレポート)

以下は、セッションを{{batchCount}}個のバッチに分けて並列に分析した結果です：

{{batchReports}}

---

## あなたのタスク

上記の{{batchCount}}個のレポートを統合し、一つの包括的な集約レポートを作成してください。

**重要な注意事項:**
1. **情報の完全保持**: マージ前のレポートにあった情報を失わないでください。すべての重要な発見、指摘、引用を統合レポートに含めてください
2. **生の引用をそのまま引き継ぐ**: 各バッチレポートに含まれる具体的な発言の引用は、セッション番号とともにそのまま統合レポートに含めてください
3. **重複の排除**: 複数のバッチで同じ発見や意見が記載されている場合のみ、一度だけ記載してください。ただし引用は異なるセッションからのものであれば複数残してください
4. **カテゴリの再構成**: バッチごとのカテゴリを見直し、より適切な分類に再構成してください
5. **網羅性の確保**: すべてのバッチから重要な発見を漏らさず含めてください。マージ前のレポートで言及されていた論点や視点がすべて統合レポートに反映されているか確認してください
6. **一貫性の維持**: トーンやスタイルを統一し、一つのレポートとして自然に読めるようにしてください
7. **文字数について**: 情報を保持するために文字数が増えることは問題ありません。包括的で詳細なレポートを作成してください

**絶対に守るべきルール:**
- 「バッチ1では」「バッチ2の分析によると」といった、バッチ分割を示唆する表現は一切使用しないでください
- すべてのセッションを最初から一度に分析したかのように記述してください
- 引用する場合も、必ず「#」を付けたセッション番号で出典を明記してください
- Markdownフォーマットで出力してください
- HTMLタグは使用しないでください
- 太字マークダウン（アスタリスク2つ）は使用しないでください
- 前置きや説明は不要で、「# {{interviewTitle}} - 集約レポート」から始めてください
- 数量的情報や量的表現は一切含めないでください (「多くの参加者が」など禁止)
- 価値判断や評価を含む表現を避けてください (「深い洞察」など禁止)
- レトリックや主観的な修飾語を排除し、学術論文のように客観的かつ簡潔に記述してください
- {{outputLengthGuidance}}

## 出力フォーマット

# {{interviewTitle}} - 集約レポート

## まとめ

[全体を通じて見えてきた主要なテーマ、パターン、重要な発見を2段落で要約してください。]

## 仮説1: [事前に設定した仮説]

### 支持する意見
- [意見1]
- [意見2]

### 反論・異なる視点
- [反論1]
- [反論2]

### 検証結果
[この仮説に対する総合的な評価]

## 仮説2: [事前に設定した仮説]

[同様に、事前に設定した仮説ごとに章立てして検証結果を記述してください]

---

統合されたレポートを出力してください：
"""

print(f"✅ MERGE_PROMPT_TEMPLATE 読み込み完了 ({len(MERGE_PROMPT_TEMPLATE):,} 文字)")

In [ ]:
#@title 🛠️ ヘルパー関数定義

def configure_genai(api_key: str):
    """Google Generative AI の設定"""
    genai.configure(api_key=api_key)
    print("✅ Google Generative AI の設定完了")

def generate_text_with_gemini(
    model_name: str,
    prompt: str,
    temperature: float = 0.3,
    max_output_tokens: int = 64000,
    max_retries: int = 3,
    retry_delay: int = 5 # seconds
) -> Dict[str, Any]:
    """
    Google Generative AI SDK でテキスト生成
    
    Returns:
        {
            "text": str,  # 生成されたテキスト
            "usage": {    # トークン使用量
                "prompt_tokens": int,
                "completion_tokens": int,
                "total_tokens": int
            }
        }
    """
    model = genai.GenerativeModel(model_name)
    
    generation_config = genai.types.GenerationConfig(
        temperature=temperature,
        max_output_tokens=max_output_tokens,
    )
    
    for attempt in range(max_retries):
        try:
            response = model.generate_content(
                prompt,
                generation_config=generation_config
            )
            
            # トークン使用量の取得
            usage = {
                "prompt_tokens": response.usage_metadata.prompt_token_count,
                "completion_tokens": response.usage_metadata.candidates_token_count,
                "total_tokens": response.usage_metadata.total_token_count
            }
            
            return {
                "text": response.text,
                "usage": usage
            }
        except Exception as e:
            print(f"⚠️ API呼び出しでエラーが発生しました (試行 {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                print(f"   {retry_delay}秒後に再試行します...")
                time.sleep(retry_delay)
            else:
                print("❌ 最大再試行回数に達しました。処理を中断します。")
                raise e

def load_messages_csv(csv_path: str) -> pd.DataFrame:
    """messages.csv を読み込み"""
    print(f"📂 CSVファイルを読み込んでいます: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"✅ {len(df):,} 件のメッセージを読み込みました")
    print(f"📊 ユニークセッション数: {df['session_id'].nunique():,}")
    return df

def filter_sessions(
    df: pd.DataFrame,
    slug: str,
    min_messages: int = 7
) -> List[str]:
    """
    条件に合うセッションIDのリストを取得
    
    Args:
        df: messages DataFrame
        slug: インタビューのslug
        min_messages: 最小メッセージ数
    
    Returns:
        session_idのリスト（メッセージ数の降順）
    """
    # slugでフィルタリング（session_idにslugが含まれるもの）
    filtered = df[df['session_id'].str.contains(slug, na=False, regex=False)]
    
    # セッションごとのメッセージ数をカウント
    session_counts = filtered.groupby('session_id').size()
    
    # 最小メッセージ数でフィルタ
    valid_sessions = session_counts[session_counts >= min_messages]
    
    # メッセージ数の降順でソート
    sorted_sessions = valid_sessions.sort_values(ascending=False)
    
    print(f"\n📊 フィルタリング結果:")
    print(f"   総セッション数: {len(session_counts):,}")
    print(f"   有効セッション数（{min_messages}メッセージ以上）: {len(valid_sessions):,}")
    print(f"   除外されたセッション: {len(session_counts) - len(valid_sessions):,}")
    
    return sorted_sessions.index.tolist()

def get_session_data(df: pd.DataFrame, session_ids: List[str]) -> List[Dict]:
    """
    指定されたsession_idのメッセージを取得
    
    Returns:
        [
            {
                "id": "session-1",
                "sessionNumber": 1,
                "messages": [
                    {"role": "user", "content": "...", "timestamp": "..."},
                    ...
                ]
            },
            ...
        ]
    """
    sessions_data = []
    
    for idx, session_id in enumerate(session_ids):
        session_messages = df[df['session_id'] == session_id].sort_values('timestamp')
        
        messages = []
        for _, row in session_messages.iterrows():
            messages.append({
                "role": row['role'],
                "content": str(row['content']),
                "timestamp": str(row['timestamp'])
            })
        
        if len(messages) > 0:
            sessions_data.append({
                "id": session_id,
                "sessionNumber": idx + 1,
                "messages": messages
            })
    
    return sessions_data

def format_sessions_data(sessions: List[Dict]) -> str:
    """セッションデータをMarkdown形式にフォーマット"""
    formatted = []
    
    for session in sessions:
        messages_text = "\n\n".join([
            f"**{msg['role']}**: {msg['content']}"
            for msg in session['messages']
        ])
        
        formatted.append(
            f"### Session #{session['sessionNumber']}\n\n{messages_text}"
        )
    
    return "\n\n---\n\n".join(formatted)

def estimate_tokens(text: str) -> int:
    """トークン数の概算（1トークン ≈ 4文字）"""
    return len(text) // 4

def create_batches(
    session_ids: List[str],
    message_counts: Dict[str, int],
    tokens_per_batch: int,
    prompt_overhead: int
) -> List[List[str]]:
    """
    セッションをバッチに分割
    
    Args:
        session_ids: 全セッションIDのリスト（メッセージ数降順）
        message_counts: {session_id: message_count}
        tokens_per_batch: バッチあたりの目標トークン数
        prompt_overhead: プロンプトテンプレートのオーバーヘッド
    
    Returns:
        バッチのリスト [[session_id, ...], ...]
    """
    batches = []
    current_batch = []
    current_tokens = prompt_overhead
    
    for session_id in session_ids:
        message_count = message_counts.get(session_id, 0)
        estimated_session_tokens = message_count * 350  # 1メッセージあたり約350トークン
        
        if current_tokens + estimated_session_tokens > tokens_per_batch and current_batch:
            # 現在のバッチを確定し、新しいバッチを開始
            batches.append(current_batch)
            current_batch = [session_id]
            current_tokens = prompt_overhead + estimated_session_tokens
        else:
            current_batch.append(session_id)
            current_tokens += estimated_session_tokens
    
    if current_batch:
        batches.append(current_batch)
    
    print(f"\n📦 バッチ分割結果:")
    print(f"   総バッチ数: {len(batches)}")
    for i, batch in enumerate(batches):
        print(f"   バッチ {i+1}: {len(batch)} セッション")
    
    return batches

print("✅ ヘルパー関数の定義完了")

In [ ]:
#@title 🔄 処理モード関数定義

# バッチ単位のリトライ設定
BATCH_MAX_RETRIES = 2   # 合計3回試行
BATCH_RETRY_DELAY = 15  # 秒

def call_batch_with_retry(fn, label: str):
    """バッチ処理のリトライラッパー"""
    for attempt in range(BATCH_MAX_RETRIES + 1):
        try:
            return fn()
        except Exception as e:
            if attempt == BATCH_MAX_RETRIES:
                print(f"❌ {label} が {BATCH_MAX_RETRIES + 1} 回の試行後に失敗しました: {e}")
                raise
            print(f"⚠️  {label} が失敗しました (試行 {attempt + 1}/{BATCH_MAX_RETRIES + 1}): {e}")
            print(f"   {BATCH_RETRY_DELAY}秒後に再試行します...")
            time.sleep(BATCH_RETRY_DELAY)

def process_sequential_mode(
    batches: List[List[str]],
    messages_df: pd.DataFrame,
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    interview_metadata: Dict[str, str]
) -> str:
    """
    逐次処理モード: バッチを1つずつ処理
    """
    current_report = ""
    
    for batch_index, batch in enumerate(batches):
        is_first_batch = batch_index == 0
        
        print(f"\n{'='*60}")
        print(f"📦 バッチ {batch_index + 1}/{len(batches)} 処理中...")
        print(f"{'='*60}\n")
        
        # セッションデータ取得
        sessions_data = get_session_data(messages_df, batch)
        sessions_text = format_sessions_data(sessions_data)
        
        # プロンプト作成
        output_length_guidance = "Target output length: 10,000-20,000 characters per batch. Provide detailed analysis with extensive quotations."
        
        if is_first_batch:
            print("🤖 初回レポート生成中...")
            prompt = INITIAL_PROMPT_TEMPLATE
        else:
            print("🔄 レポート更新中...")
            prompt = UPDATE_PROMPT_TEMPLATE.replace('{{previousReport}}', current_report)
            prompt = prompt.replace('{{newSessionCount}}', str(len(sessions_data)))
        
        # メタデータを埋め込み
        for key, value in interview_metadata.items():
            prompt = prompt.replace(f"{{{{{key}}}}}", value)
        
        prompt = prompt.replace('{{sessionCount}}', str(len(sessions_data)))
        prompt = prompt.replace('{{sessionsData}}', sessions_text)
        prompt = prompt.replace('{{outputLengthGuidance}}', output_length_guidance)
        
        estimated_tokens = estimate_tokens(prompt)
        print(f"   推定入力トークン: {estimated_tokens:,}")
        print(f"   セッション数: {len(sessions_data)}")
        
        # API呼び出し（バッチ単位のリトライ付き）
        result = call_batch_with_retry(
            lambda: generate_text_with_gemini(
                model_name=model_name,
                prompt=prompt,
                temperature=temperature,
                max_output_tokens=max_output_tokens
            ),
            f"バッチ {batch_index + 1} LLM呼び出し (sequential)"
        )
        
        current_report = result['text']
        usage = result['usage']
        
        print(f"\n✅ バッチ {batch_index + 1} 完了!")
        print(f"   入力トークン: {usage['prompt_tokens']:,}")
        print(f"   出力トークン: {usage['completion_tokens']:,}")
        print(f"   合計トークン: {usage['total_tokens']:,}")
        print(f"   出力サイズ: {len(current_report):,} 文字")
    
    return current_report

def process_parallel_mode(
    batches: List[List[str]],
    messages_df: pd.DataFrame,
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    interview_metadata: Dict[str, str]
) -> str:
    """
    並列処理モード: 全バッチを同時処理後にマージ
    """
    print("🚀 並列処理モード: 全バッチを処理中...\n")
    
    batch_reports = []
    
    # 各バッチを処理（実際にはColabでは順次実行だが、独立して処理）
    for batch_index, batch in enumerate(batches):
        print(f"\n{'='*60}")
        print(f"📦 バッチ {batch_index + 1}/{len(batches)} 処理中...")
        print(f"{'='*60}\n")
        
        # セッションデータ取得
        sessions_data = get_session_data(messages_df, batch)
        sessions_text = format_sessions_data(sessions_data)
        
        # プロンプト作成（常にINITIALテンプレート使用）
        output_length_guidance = "Target output length: 10,000-20,000 characters per batch. Provide detailed analysis with extensive quotations."
        
        prompt = INITIAL_PROMPT_TEMPLATE
        
        # メタデータを埋め込み
        for key, value in interview_metadata.items():
            prompt = prompt.replace(f"{{{{{key}}}}}", value)
        
        prompt = prompt.replace('{{sessionCount}}', str(len(sessions_data)))
        prompt = prompt.replace('{{sessionsData}}', sessions_text)
        prompt = prompt.replace('{{outputLengthGuidance}}', output_length_guidance)
        
        estimated_tokens = estimate_tokens(prompt)
        print(f"   推定入力トークン: {estimated_tokens:,}")
        print(f"   セッション数: {len(sessions_data)}")
        
        # API呼び出し（バッチ単位のリトライ付き）
        result = call_batch_with_retry(
            lambda: generate_text_with_gemini(
                model_name=model_name,
                prompt=prompt,
                temperature=temperature,
                max_output_tokens=max_output_tokens
            ),
            f"バッチ {batch_index + 1} LLM呼び出し (parallel)"
        )
        
        batch_reports.append(result['text'])
        usage = result['usage']
        
        print(f"\n✅ バッチ {batch_index + 1} 完了!")
        print(f"   入力トークン: {usage['prompt_tokens']:,}")
        print(f"   出力トークン: {usage['completion_tokens']:,}")
        print(f"   合計トークン: {usage['total_tokens']:,}")
        print(f"   出力サイズ: {len(result['text']):,} 文字")
    
    # マージ処理
    print(f"\n{'='*60}")
    print(f"🔀 {len(batch_reports)} 個のレポートをマージ中...")
    print(f"{'='*60}\n")
    
    combined_batch_reports = "\n\n".join([
        f"## バッチ {i+1} の分析結果\n\n{report}"
        for i, report in enumerate(batch_reports)
    ])
    
    merge_output_length_guidance = "Target output length: Comprehensive report with all information from batch reports preserved. Length may exceed individual batch reports to ensure no information is lost."
    
    merge_prompt = MERGE_PROMPT_TEMPLATE
    
    # メタデータを埋め込み
    for key, value in interview_metadata.items():
        merge_prompt = merge_prompt.replace(f"{{{{{key}}}}}", value)
    
    merge_prompt = merge_prompt.replace('{{batchCount}}', str(len(batch_reports)))
    merge_prompt = merge_prompt.replace('{{batchReports}}', combined_batch_reports)
    merge_prompt = merge_prompt.replace('{{outputLengthGuidance}}', merge_output_length_guidance)
    
    estimated_merge_tokens = estimate_tokens(merge_prompt)
    print(f"   推定入力トークン: {estimated_merge_tokens:,}")
    
    # マージAPI呼び出し（バッチ単位のリトライ付き）
    merge_result = call_batch_with_retry(
        lambda: generate_text_with_gemini(
            model_name=model_name,
            prompt=merge_prompt,
            temperature=temperature,
            max_output_tokens=max_output_tokens
        ),
        "マージ LLM呼び出し"
    )
    
    final_report = merge_result['text']
    merge_usage = merge_result['usage']
    
    print(f"\n✅ マージ完了!")
    print(f"   入力トークン: {merge_usage['prompt_tokens']:,}")
    print(f"   出力トークン: {merge_usage['completion_tokens']:,}")
    print(f"   合計トークン: {merge_usage['total_tokens']:,}")
    print(f"   出力サイズ: {len(final_report):,} 文字")
    
    return final_report

print("✅ 処理モード関数の定義完了")

In [ ]:
#@title ▶️ メイン実行

start_time = time.time()

print("🚀 分析レポート生成を開始します\n")

# Google Generative AI 設定
configure_genai(GOOGLE_API_KEY)

# CSV読み込み
messages_df = load_messages_csv(MESSAGES_CSV_PATH)

# セッションフィルタリング
valid_session_ids = filter_sessions(messages_df, slug, MIN_MESSAGES)

if len(valid_session_ids) == 0:
    raise ValueError(f"❌ 条件に合うセッションが見つかりませんでした（slug: {slug}, 最小メッセージ数: {MIN_MESSAGES}）")

# メッセージ数をカウント
message_counts = messages_df[messages_df['session_id'].isin(valid_session_ids)].groupby('session_id').size().to_dict()

# バッチ分割
batches = create_batches(
    valid_session_ids,
    message_counts,
    TOKENS_PER_BATCH,
    ESTIMATED_PROMPT_OVERHEAD
)

# インタビューメタデータ（簡略版 - 実際のデータに合わせて調整してください）
interview_metadata = {
    "interviewTitle": "人工知能基本計画に対する提言",
    "interviewDescription": "日本のAI戦略に関するインタビュー",
    "interviewOverview": "AI技術の発展と政策に関する専門家の意見を収集",
    "interviewThemes": "1. AI投資\n2. 人材育成\n3. データ整備\n4. 規制とバランス",
    "interviewQuestions": "1. AI投資の優先順位について\n2. 人材育成の課題\n3. データ整備の重要性",
    "knowledgeContext": "日本のAI政策、国際比較、技術動向に関する背景知識"
}

# 処理モード選択
if mode == "parallel":
    final_report = process_parallel_mode(
        batches,
        messages_df,
        MODEL_NAME,
        TEMPERATURE,
        MAX_OUTPUT_TOKENS,
        interview_metadata
    )
else:
    final_report = process_sequential_mode(
        batches,
        messages_df,
        MODEL_NAME,
        TEMPERATURE,
        MAX_OUTPUT_TOKENS,
        interview_metadata
    )

# 結果保存
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"aggregate_report_{slug}_{timestamp}.md"

# メタデータ付きで保存
report_with_metadata = f"""---
Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
Model: {MODEL_NAME}
Slug: {slug}
SessionCount: {len(valid_session_ids)}
Mode: {mode}
Temperature: {TEMPERATURE}
MinMessages: {MIN_MESSAGES}
Memo: {memo if memo else "N/A"}
---

{final_report}
"""

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(report_with_metadata)

elapsed_time = time.time() - start_time

print("\n" + "="*60)
print("✅ 分析完了!")
print("="*60)
print(f"\n📊 統計情報:")
print(f"   総セッション数: {len(valid_session_ids):,}")
print(f"   バッチ数: {len(batches)}")
print(f"   最終レポートサイズ: {len(final_report):,} 文字")
print(f"   処理時間: {elapsed_time:.1f} 秒")
print(f"\n📁 ファイル保存: {output_filename}")
print("\n💾 ダウンロード中...")

# ファイルダウンロード
files.download(output_filename)

print("\n✨ すべて完了しました！")